# Interact with app

## Files

In [1]:
# https://stackoverflow.com/questions/8230315/how-to-json-serialize-sets
from json import dumps, loads, JSONEncoder, JSONDecoder
import pickle
import time, datetime
import json
from os import listdir

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        try:
            return {'_python_object': pickle.dumps(obj).decode('latin-1')}
        except pickle.PickleError:
            return super().default(self, obj)

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(dct['_python_object'].encode('latin-1'))
    return dct

#def save_data(data=None,id='',name='',prefix='box',path='../data/pose_boxes',time=None):
 #   _filename = id + '_' + prefix + '_' + name + '_' + str(time) + '.json'
  #  with open(path + '/' + _filename,'w',encoding='utf-8') as f:
   #     g = dumps(data, cls=PythonObjectEncoder)
    #    json.dump(g,f,ensure_ascii=False,indent=4)
    #return _filename
def save_data(filename=None,data=None,id='',name='',path='../data/pose_boxes',ts=None):
    _filename=filename
    if _filename is None:
        asctime = str(time.ctime(ts)).split(' ', 1)[1].replace(' ','-')
        _filename = id + '_' + name + '_' + str(ts) + '_' + asctime + '.json'
        
    with open(path + '/' + _filename,'w',encoding='utf-8') as f:
        g = dumps(data, cls=PythonObjectEncoder)
        json.dump(g,f,ensure_ascii=False,indent=4)
    return _filename
    
def load_data(filename='',path='../data/pose_boxes'):
    with open(path + '/' + filename, 'r') as fp:
        j = json.load(fp)
        data = loads(j, object_hook=as_python_object)
        return data

### BoxPath

In [2]:
class BasePath(object):
    def get_ctime(self):
        return time.ctime(self.time_seconds)
    def get_time(self):
        return time.localtime(self.time_seconds)   

class BoxPath(object):
    def __init__(self, filename=None,time_seconds=None,x=None,y=None,key=None):
        if filename is not None:
            splits = filename.split('_')

            # there are filenames without 'box' now
            diff = 1
            if len(splits) == 7:
                diff = 0
            
            self.time_seconds = float(splits[len(splits) - 2])
            self.y= int(splits[len(splits)-4])
            self.x= int(splits[len(splits)-3])
            self.key= splits[len(splits)-7 + diff]      

            self.key=''
            for index in range(0,len(splits) - 6 + diff):
                self.key = self.key + splits[index] + '_'
            self.key=self.key[:-1]    
            self._filename=filename
        else:
            self.time_seconds=time_seconds
            self.x=x
            self.y=y
            self.key=key
            name = 'box_Box_'+str(self.y)+'_'+str(self.x)
            self._filename = self.key + '_' + name + '_' + str(time_seconds) + '_' + self.asctime() + '_' + '.json'
        
    def ctime(self):
        return time.ctime(self.time_seconds)
    
    def time(self):
        return time.localtime(self.time_seconds)        
        
    def asctime(self):
        return self.ctime().split(' ', 1)[1].replace(' ','-')
        
    def filename(self):
        return self._filename
    
    def data(self,path=None):
        return load_data(filename=self.filename(), path=path) 

### directory info

In [3]:
def listdirectory(path=None,filter='.'):
    return [x for x in listdir(path) if not x.startswith(filter)]    
    
def get_timestamps(path=None,key=None):
    results=[]
    for filename in listdirectory(path=path):
        boxpath = BoxPath(filename=filename)
        if key==boxpath.key:
            _timestamp=boxpath.time_seconds 
            if _timestamp not in results:
                results.append(_timestamp)
    return results

def get_keys_files_directory(path=None):
    results=[]
    for filename in listdirectory(path=path):
        boxpath = BoxPath(filename=filename)
        if boxpath.key not in results:
            results.append(boxpath.key)
    return results        

def get_keys_timestamps_directory(path=None):
    keys = get_keys_files_directory(path)
    results={}
    for key in keys:
        timestamps=get_timestamps(path=path,key=key)
        results[key] = timestamps
    return results    

In [4]:
#path='../data/pose_boxes' 
#eys = get_keys_files_directory(_path)
#eys

### filenames as BoxPath

In [5]:
def list_filenames(path=None):
    results=[]
    for filename in sorted(listdirectory(path=path)):
        results.append(BoxPath(filename=filename))
    return results

def filter_filenames(list=None, key=None,time_seconds=None):
    results=[]
    for filename in list:
        if key is None or filename.key == key:
            if time_seconds is None or filename.time_seconds == float(time_seconds):
                if len(results) > 0 and results[len(results)-1].time_seconds <= filename.time_seconds:
                    results.append(filename)
                else:
                    results.insert(0,filename)
    return results

In [6]:
def _test():
    _path = '../data/pose_boxes'
    list_filenames(path=_path)
    _key = 'LJ5wabMQ4Vk'
    _ts=1639756900.9205608
    filter_filenames(list=list_filenames(path=_path), time_seconds=_ts)[5].__dict__

## Pipeline

In [7]:
def print_pipeline_info(_app2):
    _video  = _app2._tab._widgets[0]
    _crop   = _app2._tab._widgets[1]   
    _resize = _app2._tab._widgets[2]  
    print('regions:',_crop.crop_list.options)
    print('region:', _crop.crop_list.value)
    print('cropped size:', (_crop.crop_size_x.value, _crop.crop_size_y.value))
    print('crop x:', (_crop.cropx0.value, _crop.cropx1.value))
    print('crop y:', (_crop.cropy0.value, _crop.cropy1.value))

    print('resize to:', _app2._tab._widgets[2].widthw.value, _app2._tab._widgets[2].heightw.value)
    print(_crop.result)
    print(_resize.result)
    
def init_shortcuts(_app):
    global _video
    global _crop
    global _resizecd 
    global _step1
    global _step2
    _video  = _app._tab._widgets[0]
    _crop   = _app._tab._widgets[1]   
    _resize = _app._tab._widgets[2]  
    _step1 = _app._tab._widgets[3]
    _step2 = _app._tab._widgets[4]
    
def set_image_quality(app,index=5,resampling='LANCZOS'): 
    _resize = app._tab._widgets[2]  
    _resize.algorithmw.value = app._tab._widgets[2].algorithmw.options['LANCZOS']
    app._tab.displayWidget.imageselectw.value = str(index)

def init_app(app):
    # für die Initialisierung nötig
    app._tab.evaluateWidgetActionList(end=None)
    init_shortcuts(app)
    set_image_quality(app)        
    
def getVideoKeys(app=None):
    _video = app._tab._widgets[0]    
    return list(_video.sources.videos.keys())

# example: loadVideo(app=app,key=None,filename='../data/videos/single/video_BJtkzEa6Tbg.json',index=5)
def loadVideo(app=None,key=None,filename=None,index=5):
    print(filename)
    _video  = app._tab._widgets[0]
    
    if filename is not None:
        _video.video_filename.value= filename
        _video.on_video_load_button_clicked('')  
        
    if key is not None:       
        _video.video_urls.value = key
    set_image_quality(app,index=index)

def deleteVideoItem(app=None,key=None):
    _video  = app._tab._widgets[0]
    del _video.sources.videos[key]    
    
def displayAllVideos(app=None,index=0):
    _video = app._tab._widgets[0]    
    _keys = getVideoKeys(app)
    for key in _keys: 
        print(key)
        loadVideo(app=app,key=key,index=index)
        display(app._tab.displayWidget.action.evaluate()['image'])
    
# example: display_all_single_videos(app=app)
def display_all_single_videos(app=None,single_directory='../data/videos/single',index=0):
    for filename in listdir(single_directory):
        if filename.startswith('video'):
            loadVideo(app=app,index=index,filename=single_directory + '/'+ filename)
            display(app._tab.displayWidget.action.evaluate()['image'])    
    
# get a list with videos and ids of current pipeline
# example: get_video_url_id(_app_test)
# returns: video_url_id_list
# example: [('JqUREqYduHw', 'https://www.youtube.com/watch?v=JqUREqYduHw')]
def get_video_url_id(app):
    results=[]
    #app=_app_test
    index=0
    _video = app._tab._widgets[0]    
    _keys = getVideoKeys(app)
    for key in _keys: 
        loadVideo(app=app,key=key,index=index)
        _video  = app._tab._widgets[0]
        results.append((key,_video.video_url.value))    
    return results    
    
# used to add new video file to ../data/videos/single   
# parameter: video_url_id_list
# example: create_single_video_files(app,video_url_id_list=[('BJtkzEa6Tbg','https://youtu.be/BJtkzEa6Tbg')])
def create_single_video_files(app,video_url_id_list=None):
    _video = app._tab._widgets[0]   
    for result in video_url_id_list:
        _video.video_filename.value='../data/videos/empty.json'
        _video.on_video_load_button_clicked('')  
        _video.video_url.value=result[1]
        _video.add_new_video(result[1])
        _video.video_filename.value='../data/videos/single/video_'+result[0]+'.json'
        _video.on_video_save_button_clicked('')    

## boxes of data with counts

In [8]:
import time
import datetime
import ipywidgets as widgets

def generate_box_bookmarks(step_y=180, step_x=160, size=None,crop=None):
    _out=widgets.Output()
    with _out:
        for y in range(0,6):
            for x in range(0,12):
                crop.cropx0.value = x*step_x
                crop.cropx1.value = (x+1)*step_x-1
                crop.cropy0.value = y*step_y
                crop.cropy1.value = (y+1)*step_y-1
                crop.add_bookmark('Box_'+str(y)+'_'+str(x))
                
                
def get_box_data_with_counts(app=None,crop=None,image=None,x=None,y=None):
    frame={}
    name='Box_'+str(y)+'_'+str(x)
    crop.crop_list.value = name
    result=app._tab.evaluateWidgetActionList(end=None,image=image)
    count=int(result['count'])
    print(name, crop.crop_list.value,result)
    if count>0:
        ts = time.time()
        st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        frame[name]=(ts,st,count,result['score_range'],result['poses'],result['bboxes'])
    return frame        

        
def get_boxes_data_with_counts(app=None,crop=None,image=None):
    frame={}
    for y in range(0,6):
        for x in range(0,12):
            name='Box_'+str(y)+'_'+str(x)
            crop.crop_list.value = name
            result=app._tab.evaluateWidgetActionList(end=None,image=image)
            count=int(result['count'])
            if count>0:
                ts = time.time()
                st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
                frame[name]=(ts,st,count,result['score_range'],result['poses'],result['bboxes'])
    return frame        

# defines frame = what is stored for very box
def get_objectboxes_data_with_counts(app=None,crop=None,image=None):
    frame={}
    for y in range(0,6):
        for x in range(0,12):
            name='Box_'+str(y)+'_'+str(x)
            crop.crop_list.value = name
            result=app._tab.evaluateWidgetActionList(end=None,image=image)
            count=int(result['count'])
            if count>0:
                ts = time.time()
                st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
                frame[name]=(ts,st,count,result['objects'],result['inference_time'])
    return frame      

def get_and_save_boxes_with_data(app=None,crop=None,path='../data/pose_boxes'):
    _out=widgets.Output()
    with _out:
        _boxes = get_boxes_data_with_counts(app=app, crop=crop) 

    ts=time.localtime(time.time())
    
    results=[str(ts[0])+'-'+str(ts[1])+'-'+str(ts[2])+'_'+str(ts[3])+'-'+str(ts[4])+'-'+str(ts[5])]
   
    for key,value in _boxes.items():
        _id=_app._tab._widgets[0].action.id 
        _filename=save_data(data=value,id=_id, name=key, time=results, path=path)
  
        
def get_and_save_objectboxes_with_data(app=None,crop=None,path='../data/object_boxes'):

    _out=widgets.Output()
    with _out:
        _boxes = get_objectboxes_data_with_counts(app=app, crop=crop) 
    ts = time.time()
    for key,value in _boxes.items():

        _id=_app._tab._widgets[0].action.id 
        _filename=save_data(data=value,id=_id, name=key, time=ts, path=path)
        
        
def get_keys_boxes_files(directory):
    results=[]
    for filename in listdir(directory):
            if filename[0]!='.':
                splits=filename.split('_')
                if splits[0] not in results:
                    results.append(splits[0])
    return results        

###  generate_box_bookmarks_normalized

In [9]:
def load(filename=None):
        with open(filename, 'r') as fp:
            file = json.load(fp)  
            return file   

def save(filename=None,data=None):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)          

def generate_box_bookmarks_normalized(bookmarks=None,x_max=12,y_max=6):
    _out=widgets.Output()
    with _out:
        step_y = 1.0 / y_max
        step_x = 1.0 / x_max
        for y in range(0,y_max):
            for x in range(0,x_max):
                cropx0 = x*step_x
                cropx1 = (x+1)*step_x
                cropy0 = y*step_y
                cropy1 = (y+1)*step_y
                bookmarks['Box_'+str(y)+'_'+str(x)] = (cropx0,cropy0,cropx1,cropy1)
                
def create_all_box_bookmark_normalized_files(file_starts='images',single_directory='../data/images/single',result_directory='../data/images/box_bookmarks_6_12', new=True, x_max=12,y_max=6):
    for filename in listdir(single_directory):
        if filename.startswith(file_starts):
            _path = single_directory + '/' + filename
            _file = load(_path)
            for key in _file.keys():
                generate_box_bookmarks_normalized(bookmarks=_file[key]['bookmarks'],x_max=x_max,y_max=y_max)  
            _path = result_directory + '/' + filename    
            save(filename=_path,data=_file)    
                
#create_all_box_bookmark_normalized_files(x_max=6,y_max=3,result_directory='../data/images/box_bookmarks_3_6')                        
                                   
#create_all_box_bookmark_normalized_files(file_starts='video',x_max=6,y_max=3,single_directory='../data/videos/single', result_directory='../data/videos/box_bookmarks_3_6')            

In [10]:
# example: create_box_bookmark_file(app=app, filename='video_BJtkzEa6Tbg.json', new=True)
def create_box_bookmark_file(app=None, filename=None, single_directory='../data/videos/single',result_directory='../data/videos/box_bookmarks_6_12',
    new=True):

        try:
            _video  = app._tab._widgets[0]
            if new == True:
                if filename in listdir(result_directory):
                    return True, ''
            _video.video_filename.value= single_directory + '/' + filename
            _video.on_video_load_button_clicked('')  
            set_image_quality(app)
            _crop   = app._tab._widgets[1] 
            generate_box_bookmarks(size=app._imageWidget2.getImage().size, crop=_crop) 
            _video.video_filename.value= result_directory + '/' + filename
            _video.on_video_save_button_clicked('') 
            return True, ''
        except Exception as e:
            return False, 'error:' + filename + ':' + str(e)
        
# example: create_all_box_bookmark_files(app=app)
def create_all_box_bookmark_files(app=None, single_directory='../data/videos/single',result_directory='../data/videos/box_bookmarks_6_12', new=True):
    for filename in listdir(single_directory):
        if filename.startswith('video'):
            create_box_bookmark_file(app=app, filename=filename, single_directory=single_directory,result_directory=result_directory,new=new)        
        
# example: get_missing_box_bookmark_files()        
def get_missing_box_bookmark_files(single_directory='../data/videos/single',result_directory='../data/videos/box_bookmarks_6_12'):
    results=[]
    for filename in listdir(single_directory):
        if filename not in listdir(result_directory):
            results.append(filename)
    return results        

In [11]:
def get_and_save_selected_boxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12',filename=None):
        try:
            loadVideo(app=app,index=5,filename=directory + '/'+ filename)
            get_and_save_boxes_with_data(app=app,crop = app._tab._widgets[1]) 
            return True, ''
        except Exception as e:
            return False, 'error:' + filename + ':' + str(e)  

def get_and_save_all_boxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12'):
    for filename in listdirectory(directory):
        get_and_save_selected_boxes_with_data(app=app, directory=directory,filename=filename)

        
        
def get_and_save_selected_objectboxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12',filename=None):
        try:
            loadVideo(app=app,index=5,filename=directory + '/'+ filename)
            get_and_save_objectboxes_with_data(app=app,crop = app._tab._widgets[1]) 
            return True, ''
        except Exception as e:
            return False, 'error:' + filename + ':' + str(e)  

def get_and_save_all_objectboxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12'):
    for filename in listdirectory(directory):
        get_and_save_selected_objectboxes_with_data(app=app, directory=directory,filename=filename)
        
        

def get_and_save_selected_bothboxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12',filename=None):
        try:
            loadVideo(app=app,index=5,filename=directory + '/'+ filename)
            get_and_save_boxes_with_data(app=app,crop = app._tab._widgets[1]) 
            get_and_save_objectboxes_with_data(app=app,crop = app._tab._widgets[1]) 
            return True, ''
        except Exception as e:
            return False, 'error:' + filename + ':' + str(e)  

def get_and_save_all_bothboxes_with_data(app=None, directory='../data/videos/box_bookmarks_6_12'):
    for filename in listdirectory(directory):
        get_and_save_selected_bothboxes_with_data(app=app, directory=directory,filename=filename)